In [10]:
#install
import pybamm
import pybammeis
import matplotlib.pyplot as plt
import numpy as np
from copy import deepcopy
# Set seed
np.random.seed(0)

In [64]:
# Define a class containing the DFN model or PMe model with degradation mechanisms(SEI growth & lithium plating and particle cracking)
class simul():
    def __init__(self, profile, cycles):
        self.exp = profile

        self.model = pybamm.lithium_ion.DFN(
            {
            "thermal": "isothermal",
            "SEI": "solvent-diffusion limited",
            "SEI porosity change": "true",
            "lithium plating": "partially reversible",
            "lithium plating porosity change": "true",  # alias for "SEI porosity change"
            "particle mechanics": ("swelling and cracking","swelling only"),
            "SEI on cracks": "true",
                "loss of active material": "stress-driven",
                "calculate discharge energy": "true",  # for compatibility with older PyBaMM versionsg
            })
        
        self.aged_model = deepcopy(self.model)

        self.param = pybamm.ParameterValues("OKane2022")
        self.param.update({
            "Initial temperature [K]": 278.15, #change the temperature 
            "SEI growth activation energy [J.mol-1]": 37000.0,
            "Ambient temperature [K]": 293.15,},)

        # Discretization points
        self.var_pts = {
            "x_n": 5,
            "x_s": 5,
            "x_p": 5,
            "r_n": 8,
            "r_p": 8,
        }
        self.solver = pybamm.IDAKLUSolver(atol=1e-5, rtol=1e-5)
        self.cycle_number = cycles  # for  aging

        self.data = {} #filled after the simulation, used to evaluate fitness

    def run(self):
        solutions = []
        # Run 1 full charge/discharge/hold cycle at a time
        for cycle in range(1, self.cycle_number + 1):
            print(f"\n Running cycle {cycle}...")

            # Run the cycle using the aged model 
            sim = pybamm.Simulation(
                self.aged_model,
                parameter_values=self.param,
                experiment=self.exp,
                solver=self.solver,
                var_pts=self.var_pts,
            )
            sol = sim.solve(calc_esoh=True,initial_soc=1)
            if cycle in [1, self.cycle_number]:
                solutions.append(sol)
        
        # Extract relevant info to later evaluate the model
        self.data['SOH'] = solutions[1].summary_variables["Capacity [A.h]"][-1] / solutions[0].summary_variables["Capacity [A.h]"][-1]

        # Now let's get the time elapsed for a complete charge
        # Consider only the first one, greedy approach that mirrors the request
        self.data['time'] = solutions[0]["Time [s]"].entries[-1] - solutions[0]["Time [s]"].entries[0]

        # Compute energy loss during the charging process
        # Use numpy to calculate energy input and output
        current = solutions[0]["Current [A]"].entries
        voltage = solutions[0]["Voltage [V]"].entries
        time = solutions[0]["Time [s]"].entries

        # Calculate energy input using numpy dot product
        energy_input = np.dot(voltage, np.diff(time, prepend=0) * current) / 3600  # Convert to Wh

        # Calculate energy output
        final_capacity = solutions[0].summary_variables["Capacity [A.h]"][-1]
        final_voltage = voltage[-1]
        energy_output = final_capacity * final_voltage  # Energy stored in the battery (Wh)

        # Calculate the energy loss
        print(f'{energy_input} , {energy_output}')
        energy_loss = energy_input - energy_output

        # Store energy loss and efficiency in the data dictionary
        self.data['energy_loss'] = round(energy_loss, 4)
        self.data['efficiency'] = (energy_output / energy_input) * 100  # Charging efficiency in percentage

    def evaluate_fitness(profile=None):
        if profile.lower == 'fast charge':
            return

        if profile == 'sustainable':
            return

        if profile == 'long life':
            return 

        # Here profile can only be all day    
        return

In [65]:
# Define what is needed to do genetic algorithm to find best profiles
es = pybamm.Experiment([
    (
        "Discharge at 1C until 2.5 V",
        "Charge at 5 A until 4.2 V",
        "Hold at 4.2 V until C/100",
        "Rest for 30 minutes",
    )
],period="30 minutes",) #change this for good accuracy

test = simul(es, 4)
test.run()
print(test.data)


 Running cycle 1...

 Running cycle 2...

 Running cycle 3...

 Running cycle 4...
-1.8490286431347678 , 21.39381941408462
{'SOH': 1.0, 'time': 12045.055970127454, 'energy_loss': -23.2428, 'efficiency': -1157.0301787112617}


In [ ]:
profiles = ['fast charge', 'sustainable', 'long life', 'all day']

# Genetic stuff
def random_profile():
    discharge_rate = np.random.choice([0.5, 1, 2])  # C-rates
    charge_current = np.random.uniform(1, 5)
    rest_time = np.random.choice([15, 30, 45, 60])  # in minutes
    hold_cutoff = np.random.choice(["C/100", "C/50"])

    return [
        f"Discharge at {discharge_rate}C until 2.5 V",
        f"Charge at {charge_current:.2f} A until 4.2 V",
        f"Hold at 4.2 V until {hold_cutoff}",
        f"Rest for {rest_time} minutes",
    ]